In [2]:
import pandas as pd

In [102]:
gpt_path = r"C:\Users\amoog\Desktop\Project_X\Project_X\gpt_msg_df.csv"
gpt_df = pd.read_csv(gpt_path, encoding = 'utf-8')
gpt_df

,Unnamed: 0,Username,Message,Timestamp,ChatGPT_Response
0,0,Waxui Alerts 🍭,AMD Day Trade idea Bit of an early Lotto idea ...,2024-01-24T22:46:10.450000+00:00,Neither No Ticker Found. The message does not ...
1,1,Waxui Alerts 🍭,Closed SNOW hereSolid catch,2023-07-14T14:05:13.101000+00:00,No Ticker Found
2,2,Waxui Alerts 🍭,Chip Chop Sitting back,2023-12-15T17:20:50.715000+00:00,No Ticker Found
3,3,Market Bishop,I'm entering,2024-03-22T16:23:53.904000+00:00,No Ticker Found
4,4,waysotheraccount,watch jd test that high of day,2023-07-13T15:51:16.112000+00:00,No Ticker Found
...,...,...,...,...,...
5884,5884,waxui,BULLPRINTER Planned Plays are posted above^ Pe...,2023-12-04T00:51:05.771000+00:00,No Ticker Found
5885,5885,waysotheraccount,taking some some tlry 2 c lottos at 17/18 for ...,2023-07-27T14:54:50.002000+00:00,Buy TLRY. The user is taking some lotto positi...
5886,5886,MEE6,Good work <@395362132432257025> you just advan...,2023-08-24T23:38:40.328000+00:00,No Ticker Found
5887,5887,Waxui Alerts 🍭,COST Swing idea Of course still keeping en eye...,2023-06-11T20:35:23.205000+00:00,Neither No Ticker Found


In [103]:
def return_stock(message):
    for substring in ['!', '.', ':', ',', '\r', '\n', '*','_','$','[',']','None','0DTE','DD']:
        message = message.replace(substring, '')
    if len(message.split())>1:
        message = message.split()[1]
    if message.strip() in ['Buy','-','Ticker'] or len(message.strip()) > 5:
        return ''
    return message

In [104]:
def buy_or_sell(message):
    return message.split()[0]

In [105]:
gpt_df['Stock'] = gpt_df['ChatGPT_Response'].apply(lambda x: return_stock(x))
gpt_df['Buy or Sell'] = gpt_df['ChatGPT_Response'].apply(lambda x: buy_or_sell(x))

stock_df = gpt_df[(gpt_df['Stock']!='') & (gpt_df['Buy or Sell'] != 'Neither')]


In [108]:
len(stock_df)

2335